# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    G=nx.Graph()
    gfile=open(infile)
    gfile.readline()
    line=gfile.readline()
    num_nodes=int(line.strip().split(' ')[2])
    nodes=range(1,num_nodes+1)
    for i in range(0,len(nodes)):
        nodes[i]=str(nodes[i])

    G.add_nodes_from(nodes)

    line=gfile.readline()

    while line.strip().split(' ')[0]=='n':
        demand_node=line.strip().split(' ')[1]
        G.node[demand_node]['demand']=int(line.strip().split(' ')[2])
        line=gfile.readline()

    edge_data=[]
    while line.strip().split(' ')[0]!="":
        array=line.strip().split(' ')
        node1=array[1]
        node2=array[2]
        cap=array[4]
        cost=array[5]
        edge_data.append([node1,node2,cap,cost,0])
        line=gfile.readline()

    G = nx.DiGraph(G)

    for i in range(0,len(edge_data)):
        if edge_data[i][4]==0:
            node1=edge_data[i][0]
            node2=edge_data[i][1]
            same_edges=[edge_data[i]]
            G.add_edge(node1,node2)

            for j in range(i+1,len(edge_data)):
                if edge_data[j][0]==node1 and edge_data[j][1]==node2:
                    same_edges.append(edge_data[j])
                    edge_data[j][4]=1
            if len(same_edges)==1:
                G.edge[node1][node2]['capacity']=int(same_edges[0][2])
                G.edge[node1][node2]['weight']=int(same_edges[0][3])
            else:
                for k in range(0,len(same_edges)):
                    if k!=0:
                        G.add_edge(node1+'_'+node2+'_'+str(k),node2+'_'+node1+'_'+str(k))#adding edges
                        G.edge[node1+'_'+node2+'_'+str(k)][node2+'_'+node1+'_'+str(k)]['capacity']=int(same_edges[k][2])
                        G.edge[node1+'_'+node2+'_'+str(k)][node2+'_'+node1+'_'+str(k)]['weight']=int(same_edges[k][3])
                    else:
                        G.edge[node1][node2]['capacity']=int(same_edges[k][2])
                        G.edge[node1][node2]['weight']=int(same_edges[k][3])
                    if k!=len(same_edges)-1 and k!=0:
                        G.add_edge(node1+'_'+node2+'_'+str(k),node1+'_'+node2+'_'+str(k+1))
                        G.add_edge(node1+'_'+node2+'_'+str(k+1),node1+'_'+node2+'_'+str(k))
                        G.add_edge(node2+'_'+node1+'_'+str(k),node2+'_'+node1+'_'+str(k+1))
                        G.add_edge(node2+'_'+node1+'_'+str(k+1),node2+'_'+node1+'_'+str(k))
                    elif k!=len(same_edges)-1 and k==0:
                        G.add_edge(node1,node1+'_'+node2+'_'+str(k+1))
                        G.add_edge(node1+'_'+node2+'_'+str(k+1),node1)
                        G.add_edge(node2,node2+'_'+node1+'_'+str(k+1))
                        G.add_edge(node2+'_'+node1+'_'+str(k+1),node2)
    
    return G
    pass

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_60 = create_graph('gte_bad.60')
#G_6830 = create_graph('gte_bad.6830')
#G_176280 = create_graph('gte_bad.176280')
print nx.min_cost_flow_cost(G_60)
#print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
#print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
#print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

94643745216


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    prob = pulp.LpProblem("Min_cost_flow",pulp.LpMinimize)
    var1=[]
    var2=[]
    cap_dict1={}
    cap_dict2={}
    weight_dict1={}
    weight_dict2={}

    for i in range(0,len(G.edges())):
        if G.edge[G.edges()[i][0]][G.edges()[i][1]].values()==[]:
            var1.append(G.edges()[i][0]+'#'+G.edges()[i][1])
        else:
            var2.append(G.edges()[i][0]+'#'+G.edges()[i][1])
            cap_dict2[G.edges()[i][0]+'#'+G.edges()[i][1]]=G.edge[G.edges()[i][0]][G.edges()[i][1]]['capacity']
            weight_dict2[G.edges()[i][0]+'#'+G.edges()[i][1]]=G.edge[G.edges()[i][0]][G.edges()[i][1]]['weight']

    var_final=var1+var2
    variables1 = pulp.LpVariable.dicts("Flows",var1,0,None,pulp.LpInteger)
    variables2={}
    for i in range(0,len(var2)):
        variables2[var2[i]]=pulp.LpVariable("flows_"+var2[i],0,cap_dict2[var2[i]],pulp.LpInteger)

    variables_final=variables1
    variables_final.update(variables2)

    prob += pulp.lpSum([weight_dict2[i]*variables2[i] for i in var2])
    demand_dict={}
    for nodes in G.nodes():
        if G.node[nodes].values()!=[]:
            demand_dict[nodes]=G.node[nodes]['demand']
        else:
            demand_dict[nodes]=0

    coef_dict={}
    for nodes in G.nodes():
        for i in var_final:
            coef_dict[i]=0
        for j in range(0,len(G.out_edges(nodes))):
            coef_dict[G.out_edges(nodes)[j][0]+'#'+G.out_edges(nodes)[j][1]]=-1
        for j in range(0,len(G.in_edges(nodes))):
            coef_dict[G.in_edges(nodes)[j][0]+'#'+G.in_edges(nodes)[j][1]]=1
        prob += pulp.lpSum([coef_dict[i]*variables_final[i] for i in var_final])==demand_dict[nodes]

    prob.solve()
    return pulp.value(prob.objective)
    pass

The following will check that the LP finds the same optimal values as previously.

In [5]:
print lp_flow_value(G_60)
#print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
#print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
#print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

94643745216.0
